In [ ]:
from google.colab import files



In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [ ]:
uploadedSqli = files.upload()

Saving SQL_Dataset1.csv to SQL_Dataset1.csv


In [ ]:
uploadedXss = files.upload()

Saving XSS_dataset.csv to XSS_dataset.csv


In [ ]:
uploadedCI = files.upload()

Saving Command_Injection_Dataset .csv to Command_Injection_Dataset .csv


In [ ]:
import pandas as pd
import io

dfSqli = pd.read_csv(io.BytesIO(uploadedSqli['SQL_Dataset1.csv']), engine='python', encoding= 'unicode_escape')
print(dfSqli)

print("______________________________________________________________\n"
      "______________________________________________________________\n ")

dfXss = pd.read_csv(io.BytesIO(uploadedXss['XSS_dataset.csv']), engine='python')
print(dfXss)

print("______________________________________________________________\n"
      "______________________________________________________________\n ")

dfCi = pd.read_csv(io.BytesIO(uploadedCI['Command_Injection_Dataset .csv']), engine='python')
print(dfCi)


                                                Sentence  Label
0                      " or pg_sleep  (  __TIME__  )  --      1
1      create user name identified by pass123 tempora...      1
2       AND 1  =  utl_inaddr.get_host_address   (    ...      1
3       select * from users where id  =  '1' or @ @1 ...      1
4       select * from users where id  =  1 or 1#"  ( ...      1
...                                                  ...    ...
30914              DELETE FROM door WHERE grow = 'small'      0
30915                               DELETE FROM tomorrow      0
30916                       SELECT wide ( s )  FROM west      0
30917       SELECT * FROM  ( SELECT slide FROM breath )       0
30918                           SELECT TOP 3 * FROM race      0

[30919 rows x 2 columns]
______________________________________________________________
______________________________________________________________
 
       Unnamed: 0                                           Sentence  Label
0 

In [ ]:
# count the dataset size sql
label_countsSqli = dfSqli['Label'].value_counts()
print(label_countsSqli)

label_countXss = dfXss['Label'].value_counts()
print(label_countXss)

label_countsCi = dfCi['Label'].value_counts()
print(label_countsCi)

0    19537
1    11382
Name: Label, dtype: int64
1    7373
0    6313
Name: Label, dtype: int64
1    470
0    255
Name: Label, dtype: int64


In [ ]:
#preprocessing
text_dataSqli = dfSqli['Sentence'].astype(str)
labelsSqli = dfSqli['Label']

text_dataXss = dfXss['Sentence'].astype(str)
labelsXss = dfXss['Label']

text_dataCi = dfCi['Sentence'].astype(str)
labelsCi = dfCi['Label']


from sklearn.model_selection import train_test_split
#split data into train and test sets
X_trainSqli, X_testSqli, y_trainSqli, y_testSqli = train_test_split(text_dataSqli, labelsSqli, test_size=0.2, random_state=42)
X_trainXss, X_testXss, y_trainXss, y_testXss = train_test_split(text_dataXss, labelsXss, test_size=0.2, random_state=42)
X_trainCi, X_testCi, y_trainCi, y_testCi = train_test_split(text_dataCi, labelsCi, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenization and word embedding
max_sequence_length = 100  # Maximum sequence length
max_num_words = 1000  # Maximum number of unique words

tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(X_trainSqli)
tokenizer.fit_on_texts(X_trainXss)
tokenizer.fit_on_texts(X_trainCi)


In [ ]:
import json

# Save the tokenizer's word-to-index mapping to a JSON file
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w', encoding='utf-8') as json_file:
    json.dump(tokenizer_json, json_file, ensure_ascii=False, indent=4)

In [ ]:
#Convert text to sequences
X_train_sequencesSqli = tokenizer.texts_to_sequences(X_trainSqli)
X_test_sequenceSqli = tokenizer.texts_to_sequences(X_testSqli)

X_train_sequencesXss = tokenizer.texts_to_sequences(X_trainXss)
X_test_sequenceXss = tokenizer.texts_to_sequences(X_testXss)

X_train_sequencesCi = tokenizer.texts_to_sequences(X_trainCi)
X_test_sequenceCi = tokenizer.texts_to_sequences(X_testCi)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Pad sequences to the same length
X_train_paddedSqli = pad_sequences(X_train_sequencesSqli, maxlen=max_sequence_length)
X_test_paddedSqli = pad_sequences(X_test_sequenceSqli, maxlen=max_sequence_length)

X_train_paddedXss = pad_sequences(X_train_sequencesXss, maxlen=max_sequence_length)
X_test_paddedXss = pad_sequences(X_test_sequenceXss, maxlen=max_sequence_length)

X_train_paddedCi = pad_sequences(X_train_sequencesCi, maxlen=max_sequence_length)
X_test_paddedCi = pad_sequences(X_test_sequenceCi, maxlen=max_sequence_length)

In [ ]:
from sklearn.preprocessing import LabelEncoder

#Encoding labels
label_encoder = LabelEncoder()

y_train_encodedSqli = label_encoder.fit_transform(y_trainSqli)
y_test_encodedSqli = label_encoder.transform(y_testSqli)

y_train_encodedXss = label_encoder.fit_transform(y_trainXss)
y_test_encodedXss = label_encoder.transform(y_testXss)

y_train_encodedCi = label_encoder.fit_transform(y_trainCi)
y_test_encodedCi = label_encoder.transform(y_testCi)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# LSTM model configuration
embedding_dim = 100   # Dimensionality of the word embeddings
lstm_units = 128    # Number of LSTM units

# Create the LSTM model
modelSqli = Sequential()
modelSqli.add(Embedding(input_dim=max_num_words, output_dim=embedding_dim, input_length=max_sequence_length))
modelSqli.add(LSTM(units=lstm_units))
modelSqli.add(Dense(1, activation = 'sigmoid'))   # Binary classfication, change activation to 'softmax' for multi-class

modelXss = Sequential()
modelXss.add(Embedding(input_dim=max_num_words, output_dim=embedding_dim, input_length=max_sequence_length))
modelXss.add(LSTM(units=lstm_units))
modelXss.add(Dense(1, activation = 'sigmoid'))   # Binary classfication, change activation to 'softmax' for multi-class

modelCi = Sequential()
modelCi.add(Embedding(input_dim=max_num_words, output_dim=embedding_dim, input_length=max_sequence_length))
modelCi.add(LSTM(units=lstm_units))
modelCi.add(Dense(1, activation = 'sigmoid'))   # Binary classfication, change activation to 'softmax' for multi-class

# Compile the model
modelSqli.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelXss.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelCi.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
batch_size = 32
epochs = 9

#Sqli
print("SQLI train")
modelSqli.fit(X_train_paddedSqli, y_train_encodedSqli, batch_size=batch_size, epochs=epochs, validation_data=(X_test_paddedSqli, y_test_encodedSqli))
print("\n_____________________________________________________________________________________________________________________"
      "_____________________________________________________________________________________________________________________")

#XSS
print("XSS train")
modelXss.fit(X_train_paddedXss, y_train_encodedXss, batch_size=batch_size, epochs=epochs, validation_data=(X_test_paddedXss, y_test_encodedXss))
print("\n_____________________________________________________________________________________________________________________"
      "_____________________________________________________________________________________________________________________")

#Ci
print("Command Injection train")
modelCi.fit(X_train_paddedCi, y_train_encodedCi, batch_size=25, epochs=9, validation_data=(X_test_paddedCi, y_test_encodedCi))

SQLI train
Epoch 1/9
773/773 [==============================] - 32s 33ms/step - loss: 0.0729 - accuracy: 0.9771 - val_loss: 0.0326 - val_accuracy: 0.9929
Epoch 2/9
773/773 [==============================] - 8s 11ms/step - loss: 0.0245 - accuracy: 0.9946 - val_loss: 0.0319 - val_accuracy: 0.9937
Epoch 3/9
773/773 [==============================] - 8s 10ms/step - loss: 0.0215 - accuracy: 0.9954 - val_loss: 0.0298 - val_accuracy: 0.9942
Epoch 4/9
773/773 [==============================] - 6s 8ms/step - loss: 0.0200 - accuracy: 0.9955 - val_loss: 0.0285 - val_accuracy: 0.9951
Epoch 5/9
773/773 [==============================] - 7s 9ms/step - loss: 0.0183 - accuracy: 0.9960 - val_loss: 0.0270 - val_accuracy: 0.9947
Epoch 6/9
773/773 [==============================] - 7s 9ms/step - loss: 0.0168 - accuracy: 0.9965 - val_loss: 0.0264 - val_accuracy: 0.9943
Epoch 7/9
773/773 [==============================] - 7s 9ms/step - loss: 0.0162 - accuracy: 0.9964 - val_loss: 0.0253 - val_accuracy: 0.994

In [ ]:
# Save the entire model as a `.keras` zip archive.
modelSqli.save('modelSqli.h5')
modelXss.save('modelXss.h5')
modelCi.save('modelCi.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf

# save trained model
modelSqli.save('/content/modelSqli')
modelXss.save('/content/modelXss')
modelCi.save('/content/modelCi')

In [ ]:
!zip -r /content/content.zip /content


  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2023.08.01/ (stored 0%)
  adding: content/.config/logs/2023.08.01/13.35.32.601305.log (deflated 58%)
  adding: content/.config/logs/2023.08.01/13.35.00.166250.log (deflated 86%)
  adding: content/.config/logs/2023.08.01/13.34.35.731641.log (deflated 58%)
  adding: content/.config/logs/2023.08.01/13.35.07.431157.log (deflated 58%)
  adding: content/.config/logs/2023.08.01/13.34.09.691819.log (deflated 91%)
  adding: content/.config/logs/2023.08.01/13.35.33.389233.log (deflated 56%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 23%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/conf

In [ ]:
#LSTM model accuracy
loss, accuracy = modelSqli.evaluate(X_test_paddedSqli, y_test_encodedSqli)
print("Sqli accuracy")
print("Loss:", loss)
print("Accuracy:", accuracy)

#XSS
loss, accuracy = modelXss.evaluate(X_test_paddedXss, y_test_encodedXss)
print("\nXss accuracy")
print("Loss:", loss)
print("Accuracy:", accuracy)

#Ci
loss, accuracy = modelCi.evaluate(X_test_paddedCi, y_test_encodedCi)
print("\nCommand injection accuracy")
print("Loss:", loss)
print("Accuracy:", accuracy)

194/194 [==============================] - 1s 5ms/step - loss: 0.0239 - accuracy: 0.9956
Sqli accuracy
Loss: 0.023869292810559273
Accuracy: 0.9956339001655579
86/86 [==============================] - 0s 4ms/step - loss: 0.0195 - accuracy: 0.9963

Xss accuracy
Loss: 0.019474446773529053
Accuracy: 0.9963477253913879
5/5 [==============================] - 0s 5ms/step - loss: 0.2177 - accuracy: 0.9034

Command injection accuracy
Loss: 0.21768788993358612
Accuracy: 0.9034482836723328


In [ ]:
# For Testing Purpose
# Input text (random string)
input_text = 'and (select substring(@@version,1,1))'  #input any payload to see wether the DL model able to classfied it or not.

# Preprocess the input text (tokenize and pad)
input_sequence = tokenizer.texts_to_sequences([input_text])
input_padded = pad_sequences(input_sequence, maxlen=max_sequence_length)

# Perform SQLi detection using the loaded model
sqli_prediction = modelSqli.predict(input_padded)[0][0]

# Perform XSS detection using the loaded model
xss_prediction = modelXss.predict(input_padded)[0][0]

# Perform XSS detection using the loaded model
ci_prediction = modelCi.predict(input_padded)[0][0]

print("sqliPrediction :", sqli_prediction)
print("XssPrediction :", xss_prediction)
print("CiPrediction :", ci_prediction)

# Classify as SQLi or XSS or Command Injection
print("\n")
if sqli_prediction > 0.5 or xss_prediction > 0.5 or ci_prediction > 0.5:
  if sqli_prediction > xss_prediction and sqli_prediction > ci_prediction:
    print("SQLi attack detected.")
  if xss_prediction > sqli_prediction and xss_prediction > ci_prediction:
    print("XSS attack detected.")
  if ci_prediction > sqli_prediction and ci_prediction > xss_prediction:
    print("Command injection attack detected")
else:
  print("No attack detected (Benign).")


1/1 [==============================] - 0s 21ms/step
sqliPrediction : 0.99999416
XssPrediction : 0.87687117
CiPrediction : 0.96420646


SQLi attack detected.


In [ ]:
# import os

# print(os.getcwd())


In [ ]:
# from tensorflow.keras.models import load_model

# # Load the saved model
# model = load_model('LSTM model.h5')
